In [3]:
import torch

In [22]:
pred = [
    [   # Sentence 0
        [0.1, 0.2, 0.3, ], # Token 0
        [0.1, 0.2, 0.3, ], # Token 1
        [0.1, 0.2, 0.3, ], # Token 2
        [0.1, 0.2, 0.3, ], # Token 3
        [0.1, 0.2, 0.3, ], # Token 4
    ],
    [   # Sentence 1
        [0.3, 0.4, 0.1, ], # Token 0
        [0.2, 0.3, 0.4, ], # Token 1
        [0.2, 0.3, 0.4, ], # Token 2
        [0.1, 0.2, 0.3, ], # Token 3
        [0.4, 0.1, 0.2, ], # Token 4
    ],
]
import torch
pred = torch.tensor(pred)
pred = torch.nn.Softmax(dim=2)(pred)
print(pred)
print(pred.shape)

label = [
    [], # Label Sentence 0
    0, # Label Sentence 1
]
label = torch.tensor(label)
print(label.shape)

fn = torch.nn.CrossEntropyLoss()
loss = fn(pred, label)
print(loss)


tensor([[[0.3006, 0.3322, 0.3672],
         [0.3006, 0.3322, 0.3672],
         [0.3006, 0.3322, 0.3672],
         [0.3006, 0.3322, 0.3672],
         [0.3006, 0.3322, 0.3672]],

        [[0.3420, 0.3780, 0.2800],
         [0.3006, 0.3322, 0.3672],
         [0.3006, 0.3322, 0.3672],
         [0.3006, 0.3322, 0.3672],
         [0.3907, 0.2894, 0.3199]]])
torch.Size([2, 5, 3])
torch.Size([2])


RuntimeError: Expected target size [2, 3], got [2]

In [ ]:
from transformers import BertForTokenClassification, BertModel
from data_dir import pretrained_3kmer_dir

bertForTokenClassification = BertForTokenClassification.from_pretrained(pretrained_3kmer_dir)

In [4]:
from data_dir import chr24_index_csv, chr24_fasta, labseq_dir, labseq_names
chr_indices = [chr24_index_csv]
chr_fastas = [chr24_fasta]
chr_labseq_path = ["{}/{}".format(labseq_dir, fname) for fname in [labseq_names[-1]]]
print(chr_indices)
print(chr_fastas)
print(chr_labseq_path)

['./data/genome/grch38/exon/NC_000024.10.csv']
['./data/chr/NC_000024.10.fasta']
['./data/genome/labseq/chr24.csv']


In [5]:
from data_dir import chr24_index_csv, chr24_fasta, labseq_dir, labseq_names
from data_preparation import generate_sequence_labelling
chr_indices = [chr24_index_csv]
chr_fastas = [chr24_fasta]
chr_labseq_path = ["{}/{}".format(labseq_dir, fname) for fname in [labseq_names[-1]]]
for src, fasta, target in zip(chr_indices, chr_fastas, chr_labseq_path):
    print("Generating sequential labelling for index {}, from fasta {}, to {}: {}".format(src, fasta, target, generate_sequence_labelling(src, fasta, target, do_expand=True, expand_size=512)))

Processing index ./data/genome/grch38/exon/NC_000024.10.csv, with fasta ./data/chr/NC_000024.10.fasta, to seq. labelling ./data/genome/labseq/chr24.csv, expanding [5431760/57226904]

In [1]:
from utils.data_generator import _data_generator_seq2seq
from models.seq2seq import DNABERTSeq2Seq
from data_dir import pretrained_3kmer_dir
dataloader = _data_generator_seq2seq()
model = DNABERTSeq2Seq(pretrained_3kmer_dir)

for step, batch in enumerate(dataloader):
    input_ids, attn_mask, token_type_ids, label = tuple(t for t in batch)
    pred = model(input_ids, attn_mask, token_type_ids)
    print(pred)

tensor([[[0.1085, 0.0951, 0.1004,  ..., 0.0995, 0.0978, 0.0987],
         [0.0949, 0.1173, 0.0994,  ..., 0.0985, 0.0937, 0.1008],
         [0.0960, 0.1087, 0.1006,  ..., 0.0997, 0.1011, 0.0977],
         ...,
         [0.0946, 0.1058, 0.0991,  ..., 0.0982, 0.0970, 0.1002],
         [0.1048, 0.1024, 0.0989,  ..., 0.0980, 0.1029, 0.0960],
         [0.1107, 0.0953, 0.1005,  ..., 0.0997, 0.0956, 0.1036]]],
       grad_fn=<SoftmaxBackward0>)
tensor([[[0.0971, 0.0910, 0.0996,  ..., 0.0995, 0.0960, 0.1047],
         [0.0995, 0.1060, 0.0975,  ..., 0.0974, 0.0976, 0.0919],
         [0.0982, 0.0913, 0.0992,  ..., 0.0990, 0.1081, 0.1080],
         ...,
         [0.1033, 0.1042, 0.0998,  ..., 0.0997, 0.0929, 0.0941],
         [0.1038, 0.0909, 0.0992,  ..., 0.0991, 0.1053, 0.1085],
         [0.0984, 0.1025, 0.1020,  ..., 0.1000, 0.1015, 0.1075]]],
       grad_fn=<SoftmaxBackward0>)
tensor([[[0.0976, 0.0976, 0.1002,  ..., 0.1002, 0.0983, 0.0984],
         [0.0924, 0.1063, 0.0992,  ..., 0.0992, 0.103

tensor([[0.0957, 0.1316, 0.1041,  ..., 0.0761, 0.1100, 0.1061],
        [0.0928, 0.0975, 0.0976,  ..., 0.0714, 0.1303, 0.0872],
        [0.0953, 0.1054, 0.1036,  ..., 0.0807, 0.1185, 0.0926],
        ...,
        [0.0862, 0.0936, 0.0937,  ..., 0.1239, 0.1213, 0.0837],
        [0.0955, 0.1037, 0.1038,  ..., 0.0809, 0.1166, 0.0928],
        [0.0995, 0.1367, 0.1081,  ..., 0.0790, 0.0755, 0.1102]],
       device='cuda:0', grad_fn=<UnbindBackward0>) tensor([0, 3, 4, 1, 1, 1, 1, 1, 2, 3, 4, 2, 3, 4, 1, 1, 2, 3, 7, 3, 4, 1, 1, 1,
        2, 5, 6, 4, 2, 3, 7, 3, 4, 1, 2, 5, 6, 4, 2, 5, 6, 4, 1, 2, 5, 8, 8, 8,
        6, 7, 3, 4, 1, 2, 3, 4, 2, 5, 6, 4, 2, 5, 6, 7, 3, 4, 1, 1, 2, 3, 4, 2,
        3, 4, 1, 2, 5, 6, 4, 2, 5, 8, 6, 4, 1, 1, 1, 1, 1, 2, 5, 8, 8, 8, 8, 8,
        6, 7, 3, 4, 2, 3, 4, 1, 2, 5, 6, 7, 5, 8, 8, 8, 6, 4, 2, 3, 4, 1, 1, 1,
        1, 1, 2, 5, 8, 8, 8, 6, 7, 3, 7, 5, 6, 4, 1, 1, 2, 3, 7, 5, 8, 6, 7, 3,
        7, 3, 4, 1, 1, 1, 2, 3, 7, 3, 7, 5, 6, 7, 3, 4, 1, 2, 3, 7, 3, 

In [4]:
from sequential_labelling import DNABERTSeq2Seq, train, init_adamw_optimizer
from transformers import get_linear_schedule_with_warmup
from data_dir import pretrained_3kmer_dir
import os
epoch_size = 10
warmup = 10
model = DNABERTSeq2Seq(pretrained_3kmer_dir)
optimizer = init_adamw_optimizer(model.parameters())
training_steps = len(dataloader) * epoch_size
optim_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup, num_training_steps=training_steps)
device = "cpu"
log_path = os.path.join("logs", "sample_log", "seq2seq", "log.2022-03-03.txt")
save_path = os.path.join("result", "samples", "seq2seq", "2022-03-03")
model.to(device)
model.train()
"""
Play with result.
"""
trained_model = train(model, optimizer, optim_scheduler, dataloader, 10, 2, log_path, save_path, device)

100%|██████████| 4/4 [00:12<00:00,  3.10s/it]


Some weights of the model checkpoint at pretrained\3-new-12w-0 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at pretrained\3-new-12w-0 and are new

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(69, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a